In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pyperclip
from selenium.webdriver.support.ui import Select
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pyperclip
from selenium.webdriver.support.ui import Select
from datetime import datetime, timedelta
import calendar

In [22]:
import datetime
import time
import pandas as pd
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import random
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pyperclip
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.proxy import Proxy, ProxyType
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import numpy as np

options = Options()
options.add_argument('--start-maximized')
options.add_experimental_option("detach", True)
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)")
service = Service(ChromeDriverManager(version="114.0.5735.90").install())
driver = webdriver.Chrome(service=service, options=options)

now = datetime.date(2023, 8, 31)
now_year_date = now
target_date = 30

arrs = ['ICN', 'CJJ', 'TAE', 'KWJ', 'PUS']
dets = ['HND']
grades_dict = {'': 'Y', 'premium': 'P', 'business': 'C', 'first': 'F'}
data = []

failed_count = 0
failed_urls = []

for gr in grades_dict:
    for arr in arrs:
        for det in dets:
            for i in range(1, target_date):
                if failed_count >= 3:
                    print("Exceeded maximum failed attempts, restarting the driver...")
                    driver.quit()
                    time.sleep(30)
                    driver = webdriver.Chrome(service=service, options=options)
                    failed_count = 0

                    # Retry the failed urls
                    if len(failed_urls) > 0:
                        print("Retrying failed urls...")
                        url1 = failed_urls[0]
                        failed_urls.pop(0)
                    else:
                        arr_date = now_year_date + datetime.timedelta(days=i)
                        url1 = f"https://flight.naver.com/flights/international/{arr}-{det}-{str(arr_date).replace('-','')}?adult=1&fareType={grades_dict[gr]}"
                else:
                    arr_date = now_year_date + datetime.timedelta(days=i)
                    url1 = f"https://flight.naver.com/flights/international/{arr}-{det}-{str(arr_date).replace('-','')}?adult=1&fareType={grades_dict[gr]}"

                driver.get(url1)
                seed = np.random.randint(100)
                np.random.seed(seed)
                a = np.random.randint(5)
                time.sleep(a)

                # Try to find the "no flight available" element first with a short timeout
                try:
                    no_flight_element = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, "//*[@id='__next']/div/div[1]/div[6]/div/div[3]/h5")))
                    if no_flight_element:
                        print(f"No flight available for: {url1}")
                        continue  # If no flight is available, move on to the next URL
                except TimeoutException:
                    pass  # If the "no flight available" element is not found, assume it's a load failure

                # If no flight is available, the code would have continued from the above statement. If the code reaches here, wait for the page to load normally.
                try:
                    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "indivisual_IndivisualItem__3co62.result")))
                    print(f"Successfully loaded the page: {url1}")
                except TimeoutException:
                    print(f"Failed to load the page: {url1}")
                    failed_count += 1
                    if len(failed_urls) < 2:  # Keep only the latest 2 URLs
                        failed_urls.append(url1)
                    continue

                Z = driver.find_elements(By.CLASS_NAME, "indivisual_IndivisualItem__3co62.result")
                for i in range(len(Z)):
                    A = Z[i].text.split("\n")
                    filtered_elements_dir = [element for element in A if element.startswith(('직항', '경유'))]
                    filtered_elements_price = [element for element in A if element.endswith('원~')]
                    grade = 'economy' if gr == '' else gr

                    if len(A) in [6, 7]:
                        time_taken = filtered_elements_dir[-1].split(", ")[0]
                        time_dir = filtered_elements_dir[-1].split(", ")[1]
                        print(time_taken)
                        print(time_dir)
                        data.append([arr, det, arr_date, A[1][:-3], A[2][:-3], time_taken, time_dir, filtered_elements_price[-1], grade])
                        print(arr, det, arr_date, A[1][:-3], A[2][:-3], time_taken, time_dir, filtered_elements_price[-1], grade)
                    else:
                        time_taken = filtered_elements_dir[-1].split(", ")[0]
                        time_dir = filtered_elements_dir[-1].split(", ")[1]
                        print(time_taken)
                        print(time_dir)
                        data.append([arr, det, arr_date, A[1][:-3], A[2][:-3], time_taken, time_dir, filtered_elements_price[-1], grade])
                        print(arr, det, arr_date, A[1][:-3], A[2][:-3], time_taken, time_dir, filtered_elements_price[-1], grade)

driver.quit()
print("크롤링 끝")

Successfully loaded the page: https://flight.naver.com/flights/international/ICN-HND-20230901?adult=1&fareType=Y
직항
02시간 20분
ICN HND 2023-09-01 20:30 22:50 직항 02시간 20분 편도 268,900원~ economy
직항
02시간 20분
ICN HND 2023-09-01 21:10 23:30 직항 02시간 20분 편도 295,500원~ economy
경유 1
28시간 05분
ICN HND 2023-09-01 20:55 01:00 경유 1 28시간 05분 편도 415,800원~ economy
경유 1
13시간 20분
ICN HND 2023-09-01 11:40 01:00 경유 1 13시간 20분 편도 415,800원~ economy
경유 1
18시간 15분
ICN HND 2023-09-01 06:45 01:00 경유 1 18시간 15분 편도 415,800원~ economy
경유 1
06시간 35분
ICN HND 2023-09-01 14:05 20:40 경유 1 06시간 35분 편도 417,300원~ economy
경유 1
07시간 20분
ICN HND 2023-09-01 14:05 21:25 경유 1 07시간 20분 편도 417,300원~ economy
경유 1
08시간 20분
ICN HND 2023-09-01 14:05 22:25 경유 1 08시간 20분 편도 418,200원~ economy
경유 1
07시간 10분
ICN HND 2023-09-01 15:20 22:30 경유 1 07시간 10분 편도 427,300원~ economy
직항
02시간 20분
ICN HND 2023-09-01 20:30 22:50 직항 02시간 20분 편도 429,300원~ economy
직항
02시간 20분
ICN HND 2023-09-01 21:10 23:30 직항 02시간 20분 편도 429,800원~ economy
경유 1
07시간 50분
ICN HND 2

In [23]:
data

[['ICN',
  'HND',
  datetime.date(2023, 9, 1),
  '20:30',
  '22:50',
  '직항',
  '02시간 20분',
  '편도 268,900원~',
  'economy'],
 ['ICN',
  'HND',
  datetime.date(2023, 9, 1),
  '21:10',
  '23:30',
  '직항',
  '02시간 20분',
  '편도 295,500원~',
  'economy'],
 ['ICN',
  'HND',
  datetime.date(2023, 9, 1),
  '20:55',
  '01:00',
  '경유 1',
  '28시간 05분',
  '편도 415,800원~',
  'economy'],
 ['ICN',
  'HND',
  datetime.date(2023, 9, 1),
  '11:40',
  '01:00',
  '경유 1',
  '13시간 20분',
  '편도 415,800원~',
  'economy'],
 ['ICN',
  'HND',
  datetime.date(2023, 9, 1),
  '06:45',
  '01:00',
  '경유 1',
  '18시간 15분',
  '편도 415,800원~',
  'economy'],
 ['ICN',
  'HND',
  datetime.date(2023, 9, 1),
  '14:05',
  '20:40',
  '경유 1',
  '06시간 35분',
  '편도 417,300원~',
  'economy'],
 ['ICN',
  'HND',
  datetime.date(2023, 9, 1),
  '14:05',
  '21:25',
  '경유 1',
  '07시간 20분',
  '편도 417,300원~',
  'economy'],
 ['ICN',
  'HND',
  datetime.date(2023, 9, 1),
  '14:05',
  '22:25',
  '경유 1',
  '08시간 20분',
  '편도 418,200원~',
  'economy'],
 ['I

In [24]:
columns = ['출발지', '도착지', '날짜', '출발시간', '도착시간', '항공편', '총 소요 시간', '가격', '좌석등급']
data

[['ICN',
  'HND',
  datetime.date(2023, 9, 1),
  '20:30',
  '22:50',
  '직항',
  '02시간 20분',
  '편도 268,900원~',
  'economy'],
 ['ICN',
  'HND',
  datetime.date(2023, 9, 1),
  '21:10',
  '23:30',
  '직항',
  '02시간 20분',
  '편도 295,500원~',
  'economy'],
 ['ICN',
  'HND',
  datetime.date(2023, 9, 1),
  '20:55',
  '01:00',
  '경유 1',
  '28시간 05분',
  '편도 415,800원~',
  'economy'],
 ['ICN',
  'HND',
  datetime.date(2023, 9, 1),
  '11:40',
  '01:00',
  '경유 1',
  '13시간 20분',
  '편도 415,800원~',
  'economy'],
 ['ICN',
  'HND',
  datetime.date(2023, 9, 1),
  '06:45',
  '01:00',
  '경유 1',
  '18시간 15분',
  '편도 415,800원~',
  'economy'],
 ['ICN',
  'HND',
  datetime.date(2023, 9, 1),
  '14:05',
  '20:40',
  '경유 1',
  '06시간 35분',
  '편도 417,300원~',
  'economy'],
 ['ICN',
  'HND',
  datetime.date(2023, 9, 1),
  '14:05',
  '21:25',
  '경유 1',
  '07시간 20분',
  '편도 417,300원~',
  'economy'],
 ['ICN',
  'HND',
  datetime.date(2023, 9, 1),
  '14:05',
  '22:25',
  '경유 1',
  '08시간 20분',
  '편도 418,200원~',
  'economy'],
 ['I

In [25]:
columns = ['출발지', '도착지', '날짜', '출발시간', '도착시간', '항공편', '총 소요 시간', '가격', '좌석등급']
df = pd.DataFrame(data, columns=columns)

# DataFrame을 CSV 파일로 저장
df.to_csv('back_airline_list1.csv', index=False, encoding='utf-8-sig')

# CSV 파일을 DataFrame으로 불러오기
df2 = pd.read_csv('back_airline_list1.csv', encoding='utf-8-sig')

# DataFrame 확인
df2

,출발지,도착지,날짜,출발시간,도착시간,항공편,총 소요 시간,가격,좌석등급
0,ICN,HND,2023-09-01,20:30,22:50,직항,02시간 20분,"편도 268,900원~",economy
1,ICN,HND,2023-09-01,21:10,23:30,직항,02시간 20분,"편도 295,500원~",economy
2,ICN,HND,2023-09-01,20:55,01:00,경유 1,28시간 05분,"편도 415,800원~",economy
3,ICN,HND,2023-09-01,11:40,01:00,경유 1,13시간 20분,"편도 415,800원~",economy
4,ICN,HND,2023-09-01,06:45,01:00,경유 1,18시간 15분,"편도 415,800원~",economy
...,...,...,...,...,...,...,...,...,...
16619,ICN,HND,2023-09-03,09:05,13:45,경유 1,04시간 40분,"편도 775,700원~",first
16620,ICN,HND,2023-09-03,09:05,13:45,경유 1,04시간 40분,"편도 797,000원~",first
16621,ICN,HND,2023-09-03,12:35,21:20,경유 1,08시간 45분,"편도 3,049,200원~",first
16622,ICN,HND,2023-09-03,12:35,21:20,경유 1,08시간 45분,"편도 3,086,300원~",first


In [19]:
df

,출발지,도착지,날짜,출발시간,도착시간,항공편,총 소요 시간,가격,좌석등급
0,ICN,HND,2023-07-29,20:30,22:50,직항,02시간 20분,"편도 411,200원~",economy
1,ICN,HND,2023-07-29,15:20,22:30,경유 1,07시간 10분,"편도 436,400원~",economy
2,ICN,HND,2023-07-29,20:30,22:50,직항,02시간 20분,"편도 438,400원~",economy
3,ICN,HND,2023-07-29,16:00,22:00,경유 1,06시간 00분,"편도 439,700원~",economy
4,ICN,HND,2023-07-29,15:20,22:25,경유 1,07시간 05분,"편도 440,400원~",economy
...,...,...,...,...,...,...,...,...,...
16898,PUS,HND,2023-08-31,08:45,22:50,경유 1,14시간 05분,"편도 734,200원~",business
16899,PUS,HND,2023-08-31,12:35,21:20,경유 1,08시간 45분,"편도 1,911,100원~",business
16900,PUS,HND,2023-08-31,10:00,06:45,경유 1,20시간 45분,"편도 3,701,700원~",business
16901,PUS,HND,2023-08-31,07:50,21:50,경유 1,14시간 00분,"편도 6,297,800원~",business


In [24]:
df.to_csv('airline_back_list_.csv', encoding='utf-8-sig')

In [6]:
def price_to_int(price_str):
    
    return int(price_str.replace('편도', '').replace(',', '').replace('원~', '').strip())


df2['가격'] = df2['가격'].apply(price_to_int)


def time_to_min(time_str):

    hours, minutes = map(int, time_str.replace('시간', '').replace('분', '').split())

    return hours * 60 + minutes



def dir_to_int(dir_str):
    if dir_str =='직항':
        return 0
    elif dir_str =='경유 1':
        return 1
    else:
        return 2

df2['총 소요 시간'] = df2['총 소요 시간'].apply(time_to_min)
df2['항공편'] = df2['항공편'].apply(dir_to_int)
display(df)

,출발지,도착지,날짜,출발시간,도착시간,항공편,총 소요 시간,가격,좌석등급
0,NRT,ICN,2023-09-02,11:50,13:55,직항,02시간 05분,"편도 221,300원~",economy
1,NRT,ICN,2023-09-02,15:40,18:25,직항,02시간 45분,"편도 230,400원~",economy
2,NRT,ICN,2023-09-02,20:25,22:50,직항,02시간 25분,"편도 230,400원~",economy
3,NRT,ICN,2023-09-02,13:30,16:10,직항,02시간 40분,"편도 230,400원~",economy
4,NRT,ICN,2023-09-02,19:25,22:00,직항,02시간 35분,"편도 234,100원~",economy
...,...,...,...,...,...,...,...,...,...
69310,SPK,ICN,2023-09-28,14:30,16:20,경유 1,25시간 50분,"편도 2,198,900원~",first
69311,SPK,ICN,2023-09-28,14:30,13:00,경유 1,22시간 30분,"편도 2,238,900원~",first
69312,SPK,ICN,2023-09-29,14:30,13:00,경유 1,22시간 30분,"편도 2,198,900원~",first
69313,SPK,ICN,2023-09-30,14:30,22:10,경유 1,07시간 40분,"편도 2,198,900원~",first


In [4]:
df = pd.DataFrame(data, columns=['Arr','Det','Arr_date', 'arr_times', 'Ret_times','time_taken','time_dir','price','grade'])

In [ ]:
df2.to_csv('back_airline_list_.csv', encoding='utf-8-sig')